In [66]:
import numpy as np
import time

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

import xgboost as xg

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [37]:
class PUF():

  def __init__(self, n):
    np.random.seed(int(time.time()))
    data = np.loadtxt('weight_diff.txt')
    
    self.N = n
    self.TARGET = 0.99
    self.w = np.zeros((n+1, 1))
    self.size = -1
    self.train_time = -1
    self.eval_time = -1

    for i in range(1, n+2):
      randi_offset = np.random.randint(1, 45481)
      self.w[i-1] = data[randi_offset-1]

  def phi(self, c):
    n = c.shape[-1]
    p = np.ones(n+1)
    p[n] = 1
    for i in range(n-1, -1, -1):
      p[i] = (2 * c[0,i] - 1) * p[i+1]

    return p

  def query(self, c, w=None, debug=False):
    phi = self.phi(c)
    if w is not None:
      r = (np.dot(phi, w) > 0)
    else:
      r = (np.dot(phi, self.w) > 0)

    if debug:
      print("Phi: ", phi, "W: ", self.w, sep="\n")

    return r

  def make_samples(self, size):
    assert size > 0
    self.size = size
    c = np.random.randint(0, 2, size=(size, 1, self.w.shape[0] - 1))
    phi = np.array([self.phi(i) for i in c])
    r = np.array([self.query(i) for i in c])

    return c, phi, r

  def train(self, train_fn):
    t0 = time.process_time()

    model = train_fn()

    t1 = time.process_time()
    self.train_time = t1 - t0                             
    print("Training time:", self.train_time)
    print("Training size:", self.size)

    return model

  def eval(self, pred=None, weights=None, eval_weights=False):
    n_test = 10000
    correct = 0
    for i in range(1, n_test + 1):
      c_test = np.random.randint(0, 2, size=(1, self.N))
      r_test = self.query(c_test)

      if eval_weights:
        assert len(weights)
        r_pred = self.query(c_test, weights)
      else:
        assert pred
        c_test = self.phi(c_test).reshape(1, -1)
        r_pred = pred(c_test)
      
      correct += (r_test == r_pred)

    success_rate = correct/n_test
    print("Success rate:", success_rate)

    # If the success rate is less than 99%, a penalty time will be added
    # One second is add for each 0.01% below 99%.
    effective_training_time = self.train_time
    if success_rate < 0.99:
        effective_training_time = self.train_time + 10000*(0.99-success_rate)
    print("Effective training time:", effective_training_time)

  def info(self):
    return {'n': self.N, 'target': self.TARGET}

In [38]:
puf = PUF(64)

X, X_phi, y = puf.make_samples(10000)

In [39]:
def lr_train():
  lr = LogisticRegression()
  lr.fit(X_phi, y.ravel())
  return lr

lr = puf.train(lr_train)

puf.eval(pred=lr.predict)

Training time: 0.07705769300000043
Training size: 10000
Success rate: [0.9932]
Effective training time: 0.07705769300000043


In [45]:
def svc_train():
  svc = SVC()
  svc.fit(X_phi, y.ravel())
  return svc

svc = puf.train(svc_train)

puf.eval(pred=svc.predict)

Training time: 0.9933555380000598
Training size: 4490
Success rate: [0.9534]
Effective training time: [366.99335554]


In [46]:
def xgb_train():
  xgb = xg.XGBClassifier()
  xgb.fit(X_phi, y.ravel())
  return xgb

xgb = puf.train(xgb_train)

puf.eval(pred=xgb.predict)

Training time: 2.059269560000075
Training size: 4490
Success rate: [0.895]
Effective training time: [952.05926956]


In [65]:
def sgd_train():
  n = 64
  w0 = np.zeros((n+1, 1))  # The estimated value of w.
  for i in range(n+1):
      w0[i] = np.random.rand()*0.1 - 0.05

  eta = 0.0001  # learning rate
  for t in range(10):
      for i in range(len(y)):
          c = X[i].flatten()
          r = y[i]
          phi = X_phi[i]

          h = (np.dot(phi, w0) > 0)
          e = float(r) - h
          delta_w = eta * e * phi.reshape(n+1,1)
          w0 = w0 + delta_w

  return w0

sgd_weights = puf.train(sgd_train)

puf.eval(weights=sgd_weights, eval_weights=True)

Training time: 0.5164980439999454
Training size: 4490
Success rate: [0.9893]
Effective training time: [7.51649804]


In [71]:
def mlp_train():
  mlp = MLPClassifier(solver='lbfgs', learning_rate_init=0.0001)
  mlp.fit(X_phi, y.ravel())
  return mlp

mlp = puf.train(mlp_train)

puf.eval(pred=mlp.predict)

Training time: 0.5814883430000464
Training size: 4490
Success rate: [0.9867]
Effective training time: [33.58148834]


In [ ]:
class Data(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X.astype(np.float32))
        self.y = torch.from_numpy(y.astype(np.float32))
        self.len = self.X.shape[0]
       
    def __getitem__(self, index):
        return self.X[index], self.y[index]
   
    def __len__(self):
        return self.len

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(65, 65)
        self.a1 = nn.ReLU()
        self.fc2 = nn.Linear(65, 65)
        self.a2 = nn.ReLU()
        self.fc3 = nn.Linear(65, 1)

    def forward(self, x):
        out = self.fc1(x)
        out = self.a1(out)
        out = self.fc2(out)
        out = self.a2(out)
        out = self.fc3(out)
        return out


model = Net()
print(model)
     


batch_size = 64

# Instantiate training and test data
train_data = Data(X, y)
train_dataloader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)

learning_rate = 0.0001

loss_fn = nn.BCEWithLogitsLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
num_epochs = 100
loss_values = []

t0 = time.process_time()

for epoch in range(num_epochs):
    for X, y in train_dataloader:
        optimizer.zero_grad()
        pred = model(X)
        loss = loss_fn(pred, y)
        loss_values.append(loss.item())
        loss.backward()
        optimizer.step()
        # print("Train Loss {:.4f}".format(loss.item()))

t1 = time.process_time()

print("Training Complete\nTime: {}\nAvg. Loss: {}".format(t1-t0, sum(loss_values)/len(loss_values)))

n_test = 10000
correct = 0

model.eval()

with torch.no_grad():
  for i in range(1, n_test+1):
      c_test = np.random.randint(0, 2, size=(1, n))
      r = puf_query(c_test, w)
      c_test = torch.from_numpy(compute_phi(c_test).astype(np.float32))
      r0 = int(model(c_test) > 0)
      correct += (r==r0)

success_rate = correct/n_test
print("Success rate:", success_rate)